# Forecasting Mini-Course Sales with Gradient Boosting Frameworks

**AIM**: To predict 2022 sales data for various fictitious learning modules from different fictitious Kaggle-branded stores in different countries with sales data pertaining to 2017-2021.

### Step 1: Importing Libraries & Examining the Data

In [ ]:
!pip install --upgrade pip

In [ ]:
!apt-get install -y python-opengl
!apt install xvfb -y

In [ ]:
!pip install -q pycaret
!pip install shap

In [ ]:
!pip install holidays

In [ ]:
!pip install --force-reinstall --no-deps numpy==1.21.0

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import holidays
from pycaret.regression import setup, compare_models 
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform as randFloat
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
# Reading in the data while parsing the 'date' column in as a datetime dtype
orig_train = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv', parse_dates = ['date'])
orig_test = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv', parse_dates = ['date'])

# Looking at the training data
orig_train.head()

In [ ]:
orig_train.info()

In [ ]:
orig_train.describe()

We'll take a look at the monthly sales per year next

In [ ]:
monthly_sales = orig_train.copy()
monthly_sales['year'] = monthly_sales['date'].dt.year
monthly_sales['month'] = monthly_sales['date'].dt.month

grouped_data = monthly_sales.groupby(['year', 'month'])['num_sold'].sum().reset_index()

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=monthly_sales, x='month', y='num_sold', hue='year', ax=ax, errorbar=None, palette='bright')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.set_xlabel('Month')
ax.set_ylabel('Sales')
ax.set_title('Monthly Sales')
plt.show()

We can notice a significant drop in sales around the time of Covid (April 2020). This will impact our final model performance and so we'll adjust these values in a later step.

In [ ]:
orig_test.info()

In [ ]:
# Plotting a distribution of target variable, num_sold
plt.figure(figsize=(8,4))
sns.histplot(orig_train['num_sold'], color='b', bins=30, kde=True)
plt.title('Number Sold Distribution')

plt.show()

The target variable distribution is stongly right-skewed, later we'll use a log transformation to amend this.

### Initial Observations:
- Our train and test sets contain a mix of categorical and numerical data.
- Our target variable is strongly right-skewed
- There is a significant drop in sales around the time of Covid (April 2020)

### Covid Sales 

In [ ]:
monthly_sales_amended = monthly_sales.copy()

april_adjust = (monthly_sales_amended['year'] == 2020) & (monthly_sales_amended['month'] == 4)
adjustment_factor_april = 1.2
monthly_sales_amended.loc[april_adjust, 'num_sold'] *= adjustment_factor_april

may_adjust = (monthly_sales_amended['year'] == 2020) & (monthly_sales_amended['month'] == 5)
adjustment_factor_may = 1.05
monthly_sales_amended.loc[may_adjust, 'num_sold'] *= adjustment_factor_may

monthly_sales_amended

In [ ]:
plt.figure(figsize=(25, 10))

# Original Data
plt.subplot(1, 2, 1)
grouped_data = monthly_sales.groupby(['year', 'month'])['num_sold'].sum().reset_index()

sns.set(style="darkgrid")
ax = sns.lineplot(data=monthly_sales, x='month', y='num_sold', hue='year', errorbar=None, palette='bright')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.set_xlabel('Month')
ax.set_ylabel('Sales')
ax.set_title('Monthly Sales')

# Amended Data
plt.subplot(1, 2, 2)
grouped_data = monthly_sales.groupby(['year', 'month'])['num_sold'].sum().reset_index()

sns.set(style="darkgrid")
ax = sns.lineplot(data=monthly_sales_amended, x='month', y='num_sold', hue='year', errorbar=None, palette='bright')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.set_xlabel('Month')
ax.set_ylabel('Sales')
ax.set_title('Monthly Sales (no Covid)')

plt.show()


We'll continue by using the adjusted values for the covid period sales.

In [ ]:
train_clean = orig_train.copy()

In [ ]:
def covid_adjustment(row):
    april_adjustment_factor = 1.2
    may_adjustment_factor = 1.05
    if row['date'].year == 2020:
        if row['date'].month == 4:  # April
            return row['num_sold'] * april_adjustment_factor
        elif row['date'].month == 5:  # May
            return row['num_sold'] * may_adjustment_factor
        else:
            return row['num_sold']
    else:
        return row['num_sold']

# Apply the adjustments to the num_sold column in the train_clean df
train_clean['num_sold'] = train_clean.apply(covid_adjustment, axis=1)

## Step 2: Combining Train & Test Datasets

We'll combine both our datasets before performing transformations on our categorical and numeric data.

In [ ]:
# Combining the data

# Isolating the target variable 
target_var = train_clean['num_sold']

# Isolating the Id column - it isn't needed in the analysis but is for testing the predictions 
ids = orig_test['id']

# Dropping target_var and ids from the datasets

new_train = train_clean.drop(['id','num_sold'],axis=1)
new_test = orig_test.drop('id',axis=1)

# Creating a new df with the combined data
combined_data = pd.concat([new_train, new_test], axis=0).reset_index(drop=True)

combined_data.info()

## Step 3: Identifying Holidays & Weekends

Holidays and weekends are known to have an impact on the sales of many products around the world. To get the most from our model we will identify and highlight these time periods.

In [ ]:
# Extracting datetime objects from our date column
combined_data['year'] = combined_data['date'].dt.year
combined_data['month'] = combined_data['date'].dt.month
combined_data['day'] = combined_data['date'].dt.day
combined_data['dayofweek'] = combined_data.date.dt.dayofweek

In [ ]:
# Identifying weekends
combined_data['is_weekend'] = combined_data['date'].dt.dayofweek.isin([5, 6]).astype(int)

In [ ]:
# Identifying holidays
country_names = ['Argentina','Spain','Estonia','Japan','Canada']
# Creating a blank df to hold all country holiday info
holiday_df = pd.DataFrame()

# Iterating through country_names and creating a dataframe for each country's holidays
for country in country_names:
    country_hols = pd.DataFrame(holidays.country_holidays(country,years=range(2017,2023)).items(), columns=['date', 'name'])
    country_hols['country'] = country
    # Appending each country's holiday df to the holiday_df
    holiday_df = pd.concat([holiday_df,country_hols],axis=0, ignore_index=True)
    
# Converting date column from object to dt for merge    
holiday_df['date'] = pd.to_datetime(holiday_df['date'])

# Merging the dataframes using a left join, this way we'll keep only those dates that are holidays
combined_final = combined_data.merge(holiday_df[['date','country']], on=['date','country'],how='left', indicator=True)

# Adding a holiday column that indicates if the day is a holiday
combined_final['holiday'] = combined_final['_merge'].apply(lambda x: 1 if x=='both' else 0)
combined_final = combined_final.drop(columns='_merge', axis=1)

combined_final = combined_final[['date','country','store','product','year','month','day','dayofweek','is_weekend','holiday']]
combined_final

In [ ]:
combined_final_df = combined_final.copy()

# Isolating our country names before we transform the country column
combined_final_df['country_name'] = combined_final['country']

## Feature Transformations

In [ ]:
# Encoding Categorical Columns
le = LabelEncoder()

combined_final_df['store'] = le.fit_transform(combined_final_df['store'])
combined_final_df['product'] = le.fit_transform(combined_final_df['product'])
combined_final_df['country'] = le.fit_transform(combined_final_df['country'])

In [ ]:
# Transforming cyclic features - month and day 
def day_month_transformer(df):
    df['month_cos'] = df['month'].apply(lambda x: np.cos(x / 12 * 2 * np.pi))
    df['day_sin'] = df['day'].apply(lambda x: np.sin(x / 365 * 2 * np.pi))
    df['day_cos'] = df['day'].apply(lambda x: np.cos(x / 365 * 2 * np.pi))
    return df

combined_final_df = day_month_transformer(combined_final_df)

combined_final_df

## Step 4: Transforming the Target Variable

In [ ]:
# Transforming the target varible with logs

log_target_var = np.log(train_clean['num_sold']) 

# Plotting the new log distribution of target variable against the original
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
sns.histplot(target_var, color='b', bins=30, kde=True)
plt.title('Number Sold Distribution')

plt.subplot(1,2,2)
sns.histplot(log_target_var, color='b', bins=30, kde=True)
plt.title('Number Sold Log Distribution')

plt.show()

We now have a much better distribution of our target varible and will use this log version going forward.

## Step 5: Splitting the Data
Now, we'll split the data back into the original training and test sets.

In [ ]:
final_train = combined_final_df.iloc[:136950,:]
final_train.info()

In [ ]:
final_test = combined_final_df.iloc[136950:,:].reset_index(drop=True)
final_test.info()

In [ ]:
pycaret_setup = setup(data=(pd.concat([final_train,log_target_var],axis=1)), target='num_sold')

In [ ]:
compare_models()

## Step 6: Training the Model
Since catboost, xgb and lightgbm were the top performing models above, we will use them to make our predictions.

In [ ]:
# Creating training and validation sets from our full training data to access performance
X_train, X_val, y_train, y_val = train_test_split(final_train, log_target_var, test_size=0.3, random_state=42)

In [ ]:
# Initializing the models
catboost_model = CatBoostRegressor()
xgboost_model = XGBRegressor()
lightgbm_model = LGBMRegressor()

# Parameter distributions for RandomizedSearchCV for CBR only
catboost_param_dist = {
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': randint(100, 301),  # Random integer between 100 and 300 (inclusive)
    'depth': randint(4, 11),          # Random integer between 4 and 10 (inclusive)
    'l2_leaf_reg': [1, 3, 5, 7,9],
    'subsample' : randFloat(0.05, 0.95),
    'colsample_bylevel': randFloat(0.05, 0.95)
    
}

In [ ]:
# Creating a TimeSeriesSplit object 
tscv = TimeSeriesSplit(n_splits=5)

# RandomizedSearchCV for CBR to determine best params
random_search_catboost = RandomizedSearchCV(
    catboost_model,
    param_distributions=catboost_param_dist,
    n_iter=20,
    cv=tscv,
    n_jobs=-1
)

In [ ]:
# Fitting the RandomizedSearchCV results to the data 
features = ['country', 'store', 'product', 'year', 'month',
       'day', 'dayofweek', 'is_weekend','month_cos', 'day_sin', 'day_cos', 'holiday']

random_search_catboost.fit(X_train[features], y_train,verbose=0) 

## Step 7: Evaluating Model Performance

In [ ]:
# Getting the best parameters and score for the CBR model
best_params_catboost = random_search_catboost.best_params_
best_score_catboost = random_search_catboost.best_score_

print("\nBest CatBoost Score:", best_score_catboost)
print("Best CatBoost Parameters:", best_params_catboost)

In [ ]:
# Training all of our models on the training data
models = {}

# CBR
models["catboost"] = CatBoostRegressor(**best_params_catboost, random_state=12,verbose=False)
models['catboost'].fit(X_train[features], y_train, 
                       eval_set = (X_val[features], y_val), verbose = False)
# XGB
models['xgb'] = XGBRegressor()
models['xgb'].fit(X_train[features], y_train)

# LGBM
models['lgbm'] = LGBMRegressor()
models['lgbm'].fit(X_train[features],y_train)

In [ ]:
# Calculating the sMAPE score

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

print("sMAPE catboost:",smape(np.exp(y_val),
                              np.exp(models['catboost'].predict(X_val[features]))))
print("sMAPE xgb:",smape(np.exp(y_val),
                              np.exp(models['xgb'].predict(X_val[features]))))
print("sMAPE lightgbm:",smape(np.exp(y_val),
                              np.exp(models['lgbm'].predict(X_val[features]))))

Our sMAPE scores for each model are low, indicating a very good performance from each of the models when predicting using our validation set.

## Step 8: Predictions

In [ ]:
# Training our models for the fianl predictions
models = {}


models["catboost"] = CatBoostRegressor(**best_params_catboost, random_state=12,verbose=False)
models['catboost'].fit(final_train[features], log_target_var, 
                       verbose = False)

models['xgb'] = XGBRegressor()
models['xgb'].fit(final_train[features], log_target_var)

models['lgbm'] = LGBMRegressor()
models['lgbm'].fit(final_train[features],log_target_var)

In [ ]:
# Making predictions using an ensemble of our models
predictions = (np.exp(models['catboost'].predict(final_test[features]))*0.33+
               np.exp(models['xgb'].predict(final_test[features]))*0.33+
               np.exp(models['lgbm'].predict(final_test[features]))*0.33)

In [ ]:
# Making our predictions a series so we can add it to a DF
predictions =  pd.Series(predictions)

# Combining our predictions with the Ids we extracted at the beginning
final_predictions = pd.concat([ids,predictions], axis=1)

# Renaming our columns 
final_predictions.columns = ['Id','num_sold']

# Rounding our num_sold column values
final_predictions['num_sold'] = np.round(final_predictions['num_sold'],0)

# Looking at our final predictions
final_predictions

In [ ]:
submissions = final_predictions
submissions.to_csv('./submission.csv', index=False, header=True)